For background information, refer to the ReadMe on github
* https://github.com/wpbSabi/sustainable_supply_chain_template 

In [1]:
# !pip install -U googlemaps 
import folium
import numpy as np
import pandas as pd
import googlemaps

# Data

## Geocoding

My first choice would have been to use geopy / nominatum to obtain latitudes and longitudes based on city names. However, there was an issue on April 8, 2023 with one of the services geopy relies on.  The good news is that on April 18, 2023 the service was running again.
* https://geopy.readthedocs.io/en/stable/

In [2]:
# from geopy.geocoders import Nominatim # OpenStreetMap API for geocoding
# geolocator = Nominatim(user_agent="april_8")
# city = "Chicago"
# country = "USA"
# loc = geolocator.geocode(city + ',' + country)
# print("latitude is: ", loc.latitude, "\nlongtitude is:-", loc.longitude)

Instead, I signed up for a google maps api

In [3]:
# gmaps_key = googlemaps.Client(key='<YOUR_KEY_HERE')

# # Import or set format for data
# df = pd.read_csv('data/location_lat_long.csv')
# # Create a list, to retrieve multiple locations at a time
# list_of_cities = ['Salt Lake City, Utah',
#                   'New Orleans, Louisiana',
#                   'Hartford, Connecticut',
#                   'Buffalo, New York',
#                   'Birmingham, Alabama']
# for city in list_of_cities:
#     g = gmaps_key.geocode(city)
#     address = g[0]['formatted_address']
#     lat = g[0]['geometry']['location']['lat']
#     long = g[0]['geometry']['location']['lng']
#     df.loc[len(df), 'location'] = address
#     df.loc[len(df) - 1, 'latitude'] = lat
#     df.loc[len(df) - 1, 'longitude'] = long

In [4]:
# def map_locations(df):
#     # Verify the new data by creating a map (or look at the df)
#     m = folium.Map([40, -95],  zoom_start=4)
#     # Add orange circles
#     for i in range(len(df)): folium.CircleMarker(
#         location=[df.iloc[i]['latitude'],
#                 df.iloc[i]['longitude']],
#         tooltip=df.iloc[i]['location'],
#         color='orange',
#         fill=True,
#         fill_opacity=0.7,
#         radius=5 # df.iloc[i]['demand_fix'] #5 #df.iloc[i]['Demand FY21']
#         ).add_to(m)
#     # To enable the toggle between map base layers
#     folium.TileLayer('OpenStreetMap').add_to(m)
#     folium.TileLayer('Stamen Terrain').add_to(m)
#     folium.TileLayer('Stamen Toner').add_to(m)
#     folium.TileLayer('Stamen Water Color').add_to(m)
#     folium.TileLayer('CartoDB positron').add_to(m)
#     folium.TileLayer('CartoDB dark_matter').add_to(m)
#     folium.LayerControl().add_to(m)
#     # m.save('images/python_folium_bubble_map.html')
#     return m
# map_locations(lat_long_data)

In [5]:
# Overwrite the data file with the updates
# df.to_csv('data/location_lat_long.csv', index=False)

## Greatest Circle Distance (GCD)

In [6]:
# Import the 
lat_long_data = pd.read_csv('data/location_lat_long.csv')

# Create a data frame of the possible Origin / Destination pairs
origins = lat_long_data.copy()
destinations = lat_long_data.copy()
origin_destination_pairs = origins.merge(destinations, how='cross')

In [7]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the greatest circle distance between two points on Earth by 
        utilizing the haversine calculation via latitude and longitude data.
    """
    r = 6371
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2) ** 2 \
        + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
    return np.round(res, 2)

In [8]:
# Distance in miles from Portland to Boise, used in the greenhouse gas example
print(lat_long_data.loc[0])
print(haversine_distance(lat_long_data.loc[0, 'latitude'],
                   lat_long_data.loc[0, 'longitude'],
                   43.6150,
                   -116.2023))

location     Portland, OR, USA
latitude             45.515232
longitude          -122.678385
Name: 0, dtype: object
554.64


In [9]:
def calculate_distances(df):
    """
    Utilize haversine_distance() and the df for origin / destination pairs
    Calculate the distances between all origin / destination pairs
    """
    df['distance'] = df\
    .apply(lambda x: haversine_distance(x['latitude_x'],
                                        x['longitude_x'],
                                        x['latitude_y'],
                                        x['longitude_y']), axis=1)
    # For shipments within a city, set the distance to 10 miles
    df['distance'] = np.where(df['location_x']==df['location_y'],
                              10,
                              df['distance'])
    return df 
distance_between_od_pairs = calculate_distances(origin_destination_pairs)

## Scenarios / Greenhouse gas calculations
Create shipments for scenarios, and assumptions:
* 0.36 grams co2e per mile for parcels shipments or LTL shipments
* DC footprint - 443,000 grams co2e per day

### One Distribution Center

In [10]:
def footprint_for_one_dc(df):
    """
    Return the emissions from one daily shipment to each of the 50 cities
    Returns total emissions and transport emissions in kg co2e
    """
    co2e_per_mile_per_truck = 0.36 # grams of co2e per mile for LTL truck
    co2e_for_facility = 443000 # grams of co2e, average distribution center
    df = df.assign(co2e_shipment = df['distance'] * co2e_per_mile_per_truck)
    total_kg_co2e = round(sum(df['co2e_shipment']) + co2e_for_facility)
    transport_kg_co2e = round(sum(df['co2e_shipment']) )
    return round(total_kg_co2e / 1000), round(transport_kg_co2e / 1000)

In [11]:
def map_locations(df, dc_list):
    """
    Creates a map of origins and destinations
    df contains the destinations
    dc_list contains the distribution centers
    """
    # Verify the new data by creating a map (or look at the df)
    m = folium.Map([40, -95],  zoom_start=4)
    # Add darkgreen circles for destinations
    for i in range(len(df)): folium.CircleMarker(
        location=[df.iloc[i]['latitude'],
                df.iloc[i]['longitude']],
        tooltip=df.iloc[i]['location'],
        color='darkgreen',
        fill=True,
        fill_opacity=0.7,
        radius=5 
        ).add_to(m)
    for i in dc_list: 
        # Add lines to show Origin-Destination pairs
        add_lines_df = pd.DataFrame()
        dc = df[df['location']==i].reset_index(drop=True)
        for j in range(0, len(df)):
            origin = pd.concat([dc]*50).reset_index(drop=True)
            od = [(origin['latitude'][j], origin['longitude'][j]),
                 (df['latitude'][j], df['longitude'][j])]
            add_lines_df[j]=od
            folium.PolyLine(add_lines_df[j], color='darkgreen').add_to(m)
        # Add blue circles for distribution centers
        folium.CircleMarker(
            dc = df[df['location']==i].reset_index(drop=True),
            location=[dc.iloc[0]['latitude'],
                    dc.iloc[0]['longitude']],
            tooltip=dc.iloc[0]['location'],
            color='blue',
            fill=True,
            fill_opacity=0.3,
            radius=10
        ).add_to(m)    
    # To enable the toggle between map base layers
    folium.TileLayer('Stamen Terrain').add_to(m)
    folium.TileLayer('OpenStreetMap').add_to(m)
    folium.TileLayer('Stamen Toner').add_to(m)
    folium.TileLayer('Stamen Water Color').add_to(m)
    folium.TileLayer('CartoDB positron').add_to(m)
    folium.TileLayer('CartoDB dark_matter').add_to(m)
    folium.LayerControl().add_to(m)
    # m.save('images/python_folium_bubble_map.html')
    return m

In [12]:
# Scenario 1: Distribution Center at Memphis
scen1 = distance_between_od_pairs[
    distance_between_od_pairs['location_x']=='Memphis, TN, USA']
display(footprint_for_one_dc(scen1))
map_locations(lat_long_data, ['Memphis, TN, USA'])

(467, 24)

In [13]:
# Scenario 2: Distribution Center at St Louis
scen2 = distance_between_od_pairs[
    distance_between_od_pairs['location_x']=='St. Louis, MO, USA']
display(footprint_for_one_dc(scen2))
map_locations(lat_long_data, ['St. Louis, MO, USA'])

(466, 23)

In [14]:
# Scenario 3: Distribution Center at Seattle (probably the worst choice)
scen3 = distance_between_od_pairs[
    distance_between_od_pairs['location_x']=='Seattle, WA, USA']
display(footprint_for_one_dc(scen3))
map_locations(lat_long_data, ['Seattle, WA, USA'])

(494, 51)

In [15]:
print((494 - 466) / 494)
print((494 - 466) / 466)

0.05668016194331984
0.060085836909871244


### Three Distribution Centers

In [16]:
# Scenario 4: Riverside, Houston, and Columbus
# Distribution Center at Seattle (probably the worst choice)
# 1 daily shipment to each of the 50 cities
s4 = distance_between_od_pairs[
    (distance_between_od_pairs['location_x']=='Riverside, CA, USA') | 
    (distance_between_od_pairs['location_x']=='Houston, TX, USA') | 
    (distance_between_od_pairs['location_x']=='Columbus, OH, USA')
]

In [17]:
co2e_per_mile_per_truck = 0.36 # grams of co2e for LTL truck
co2e_per_facility_daily = round(443000 / np.sqrt(3))
print('Average facility size:', co2e_per_facility_daily)

Average facility size: 255766


In [63]:
# For each destination, use the closest facility
scen4 = s4[['location_y', 'distance']]\
            .groupby(['location_y'], as_index=False)\
            .min()

In [19]:
scen4['co2e_shipment'] = scen4['distance'] * co2e_per_mile_per_truck

# Print Total kg co2e 
print(round(round(
    sum(scen4['co2e_shipment']) + (co2e_per_facility_daily * 3)) / 1000))
# Print kg co2e from transportation
print(round(sum(scen4['co2e_shipment']) / 1000))

779
11


In [20]:
print((779 - 466) / 779)

0.4017971758664955


In [21]:
lat_long_data.head()

,location,latitude,longitude
0,"Portland, OR, USA",45.515232,-122.678385
1,"Seattle, WA, USA",47.606209,-122.332071
2,"New York, NY, USA",40.712775,-74.005973
3,"Los Angeles, CA, USA",34.052234,-118.243685
4,"Chicago, IL, USA",41.878114,-87.629798


In [22]:
map_locations(scen4.merge(lat_long_data,
                          left_on='location_y',
                          right_on='location'), 
              ['Riverside, CA, USA',
              'Houston, TX, USA',
              'Columbus, OH, USA'])

In [23]:
# map multiple DCs
mdc = scen4.merge(lat_long_data,
                          left_on='location_y',
                          right_on='location')
mdc

,location_y,distance,co2e_shipment,location,latitude,longitude
0,"Atlanta, GA, USA",701.73,252.6228,"Atlanta, GA, USA",33.748752,-84.387684
1,"Austin, TX, USA",235.35,84.7260,"Austin, TX, USA",30.267153,-97.743061
2,"Baltimore, MD, USA",551.92,198.6912,"Baltimore, MD, USA",39.290385,-76.612189
3,"Birmingham, AL, USA",792.60,285.3360,"Birmingham, AL, USA",33.518589,-86.810357
4,"Boston, MA, USA",1033.54,372.0744,"Boston, MA, USA",42.360082,-71.058880
5,"Buffalo, NY, USA",473.00,170.2800,"Buffalo, NY, USA",42.886447,-78.878369
6,"Charlotte, NC, USA",559.57,201.4452,"Charlotte, NC, USA",35.227087,-80.843127
7,"Chicago, IL, USA",443.57,159.6852,"Chicago, IL, USA",41.878114,-87.629798
8,"Cincinnati, OH, USA",161.07,57.9852,"Cincinnati, OH, USA",39.103118,-84.512020
9,"Cleveland, OH, USA",203.30,73.1880,"Cleveland, OH, USA",41.499320,-81.694361


In [64]:
scen4

,location_y,distance
0,"Atlanta, GA, USA",701.73
1,"Austin, TX, USA",235.35
2,"Baltimore, MD, USA",551.92
3,"Birmingham, AL, USA",792.60
4,"Boston, MA, USA",1033.54
5,"Buffalo, NY, USA",473.00
6,"Charlotte, NC, USA",559.57
7,"Chicago, IL, USA",443.57
8,"Cincinnati, OH, USA",161.07
9,"Cleveland, OH, USA",203.30


In [65]:
s4


,location_x,latitude_x,longitude_x,location_y,latitude_y,longitude_y,distance
300,"Houston, TX, USA",29.760427,-95.369803,"Portland, OR, USA",45.515232,-122.678385,2950.92
301,"Houston, TX, USA",29.760427,-95.369803,"Seattle, WA, USA",47.606209,-122.332071,3040.50
302,"Houston, TX, USA",29.760427,-95.369803,"New York, NY, USA",40.712775,-74.005973,2281.34
303,"Houston, TX, USA",29.760427,-95.369803,"Los Angeles, CA, USA",34.052234,-118.243685,2206.26
304,"Houston, TX, USA",29.760427,-95.369803,"Chicago, IL, USA",41.878114,-87.629798,1515.80
...,...,...,...,...,...,...,...
1595,"Columbus, OH, USA",39.961176,-82.998794,"Salt Lake City, UT, USA",40.760779,-111.891047,2438.52
1596,"Columbus, OH, USA",39.961176,-82.998794,"New Orleans, LA, USA",29.951066,-90.071532,1285.15
1597,"Columbus, OH, USA",39.961176,-82.998794,"Hartford, CT, USA",41.765804,-72.673372,890.58
1598,"Columbus, OH, USA",39.961176,-82.998794,"Buffalo, NY, USA",42.886447,-78.878369,473.00


In [69]:
# For each destination, use the closest facility
scen4_all = s4[s4['distance'].isin(scen4['distance'])].reset_index(drop=True)
scen4_all.tail()

,location_x,latitude_x,longitude_x,location_y,latitude_y,longitude_y,distance
45,"Columbus, OH, USA",39.961176,-82.998794,"Richmond, VA, USA",37.540725,-77.436048,552.24
46,"Columbus, OH, USA",39.961176,-82.998794,"Louisville, KY, USA",38.252665,-85.758456,304.59
47,"Columbus, OH, USA",39.961176,-82.998794,"Hartford, CT, USA",41.765804,-72.673372,890.58
48,"Columbus, OH, USA",39.961176,-82.998794,"Buffalo, NY, USA",42.886447,-78.878369,473.00
49,"Columbus, OH, USA",39.961176,-82.998794,"Birmingham, AL, USA",33.518589,-86.810357,792.60


In [79]:
df = scen4_all.copy()
# Verify the new data by creating a map (or look at the df)
m = folium.Map([40, -95],  zoom_start=4)
# Add darkgreen circles for destinations
for j in range(len(df)): 
    folium.CircleMarker(
        location=[df.iloc[j]['latitude_y'],
                df.iloc[j]['longitude_y']],
        tooltip=df.iloc[j]['location_y'],
        color='darkgreen',
        fill=True,
        fill_opacity=0.7,
        radius=5 
    ).add_to(m)
for i in df['location_x'].unique(): 
    # Add lines to show Origin-Destination pairs
    add_lines_df = pd.DataFrame()
    dc = df[df['location_x']==i].reset_index(drop=True)
    for k in range(0, len(df)):
        od = [(df['latitude_x'][k], df['longitude_x'][k]),
             (df['latitude_y'][k], df['longitude_y'][k])]
        add_lines_df[j]=od
        folium.PolyLine(add_lines_df[j], color='darkgreen').add_to(m)
    # Add blue circles for distribution centers
    dc = df[df['location_x']==i].reset_index(drop=True)
    folium.CircleMarker(
        location=[dc.iloc[0]['latitude_x'],
                dc.iloc[0]['longitude_x']],
        tooltip=dc.iloc[0]['location_x'],
        color='blue',
        fill=True,
        fill_opacity=0.3,
        radius=10
    ).add_to(m)    
# To enable the toggle between map base layers
folium.TileLayer('Stamen Terrain').add_to(m)
folium.TileLayer('OpenStreetMap').add_to(m)
folium.TileLayer('Stamen Toner').add_to(m)
folium.TileLayer('Stamen Water Color').add_to(m)
folium.TileLayer('CartoDB positron').add_to(m)
folium.TileLayer('CartoDB dark_matter').add_to(m)
folium.LayerControl().add_to(m)

In [80]:
m